# Playing with SQLite

The [SQLite](https://en.wikipedia.org/wiki/SQLite) is shipped with Python, we have nothing to install. Let's play with it.

A few links:
- http://zetcode.com/python/sqlite/
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html
- https://www.dataquest.io/blog/python-pandas-databases/

Pro-tip: use [sqlitebrowser](https://github.com/sqlitebrowser/sqlitebrowser) to visualize the database. This is a GUI, much more user-friendly than the SQLite shell.

Let's play with our DGEMM dataset, used in the paper.

In [1]:
!wget -c https://github.com/Ezibenroc/calibration_analysis/raw/master/dahu/blas/dgemm_calibration.csv -O /tmp/data.csv
!cut -d, -f1,2,3,4,5,6,10,11 /tmp/data.csv > /tmp/dgemm.csv
!head /tmp/dgemm.csv

--2019-06-06 18:29:40--  https://github.com/Ezibenroc/calibration_analysis/raw/master/dahu/blas/dgemm_calibration.csv
Résolution de github.com… 140.82.118.3
Connexion à github.com|140.82.118.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://media.githubusercontent.com/media/Ezibenroc/calibration_analysis/master/dahu/blas/dgemm_calibration.csv [suivant]
--2019-06-06 18:29:40--  https://media.githubusercontent.com/media/Ezibenroc/calibration_analysis/master/dahu/blas/dgemm_calibration.csv
Résolution de media.githubusercontent.com… 151.101.120.133
Connexion à media.githubusercontent.com|151.101.120.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 614553640 (586M) [text/plain]
Sauvegarde en : « /tmp/data.csv »

/tmp/data.csv       100%[===================>] 586,08M  86,9MB/s    ds 8,3s    

2019-06-06 18:29:59 (70,2 MB/s) — « /tmp/data.csv » sauvegardé [614553640/614553640]

function,m,n,k,timestamp,dura

## SQLite and Pandas, the basics

First, let's load our huge CSV file containing dgemm data and dump it into a SQLite table.

In [2]:
import pandas
import sqlite3

In [3]:
%time df = pandas.read_csv('/tmp/dgemm.csv')
print(len(df))
df.head()

CPU times: user 2.63 s, sys: 309 ms, total: 2.94 s
Wall time: 2.93 s
5004288


,function,m,n,k,timestamp,duration,node,core
0,dgemm,378,7640,2427,3473.428414,0.485947,10,0
1,dgemm,378,7640,2427,3473.914385,0.486129,10,0
2,dgemm,378,7640,2427,3474.400522,0.486853,10,0
3,dgemm,9441,640,1160,3474.887383,0.455139,10,0
4,dgemm,9441,640,1160,3475.342530,0.453528,10,0


In [4]:
%%time
connection = sqlite3.connect('/tmp/dgemm.db')
df.to_sql('mytable', connection, index=False)

CPU times: user 5.91 s, sys: 656 ms, total: 6.56 s
Wall time: 6.56 s


In [5]:
!du -sh /tmp/dgemm*

250M	/tmp/dgemm.csv
215M	/tmp/dgemm.db


Interesting, we have saved some space by using a SQLite database, but not as much as I expected. Maybe there is some tuning to do.

No, let's see how much time is needed to read from this database.

In [6]:
%time tmp = pandas.read_sql('select * from mytable', connection)
print(len(tmp))
tmp.head()

CPU times: user 9.75 s, sys: 907 ms, total: 10.7 s
Wall time: 10.7 s
5004288


,function,m,n,k,timestamp,duration,node,core
0,dgemm,378,7640,2427,3473.428414,0.485947,10,0
1,dgemm,378,7640,2427,3473.914385,0.486129,10,0
2,dgemm,378,7640,2427,3474.400522,0.486853,10,0
3,dgemm,9441,640,1160,3474.887383,0.455139,10,0
4,dgemm,9441,640,1160,3475.342530,0.453528,10,0


Wow, reading from the database is nearly twice longer than reading from the CSV file. This feels weird. Let's see if we can at least have a low time by reading a subset.

In [7]:
%time tmp = pandas.read_sql('select duration from mytable', connection)
print(len(tmp))
tmp.head()

CPU times: user 1.88 s, sys: 124 ms, total: 2 s
Wall time: 2 s
5004288


,duration
0,0.485947
1,0.486129
2,0.486853
3,0.455139
4,0.453528


In [8]:
%time tmp = pandas.read_sql('select * from mytable where node==20', connection)
print(len(tmp))
tmp.head()

CPU times: user 482 ms, sys: 58.1 ms, total: 540 ms
Wall time: 540 ms
156384


,function,m,n,k,timestamp,duration,node,core
0,dgemm,378,7640,2427,3485.569752,0.486023,20,0
1,dgemm,378,7640,2427,3486.055798,0.486281,20,0
2,dgemm,378,7640,2427,3486.542086,0.485554,20,0
3,dgemm,9441,640,1160,3487.027647,0.461661,20,0
4,dgemm,9441,640,1160,3487.489317,0.458144,20,0


Good, we see the benefit of having a database here. With the CSV file, we would have to first read the whole CSV and then filter, which would be longer (since reading the CSV takes about 7s).

In [9]:
connection.close()

## A tentative to shrink the database

Let's take **all** the parameters in their own table.

In [10]:
!rm /tmp/dgemm.db

In [11]:
def extract_parameters(df, parameters):
    df_params = df[parameters].drop_duplicates()
    df_params['parameters_id'] = range(len(df_params))
    df = df.set_index(parameters).join(df_params.set_index(parameters)).reset_index()
    df = df.drop(parameters, axis=1)
    return df, df_params

tmp_values, tmp_params = extract_parameters(df, ['function', 'm', 'n', 'k', 'node', 'core'])
print(len(tmp_values))
print(len(tmp_params))

5004288
556032


In [12]:
tmp_values.head()

,timestamp,duration,parameters_id
0,3723.065742,1.432000e-06,451899
1,3723.065746,2.530000e-07,451899
2,3723.065748,1.830000e-07,451899
3,3906.877902,1.520000e-06,451899
4,3906.877906,2.890000e-07,451899


In [13]:
tmp_params.head()

,function,m,n,k,node,core,parameters_id
0,dgemm,378,7640,2427,10,0,0
3,dgemm,9441,640,1160,10,0,1
6,dgemm,1041,2183,735,10,0,2
9,dgemm,1248,1343,1991,10,0,3
12,dgemm,642,6716,1313,10,0,4


In [14]:
%%time
connection = sqlite3.connect('/tmp/dgemm.db')
tmp_values.to_sql('exp_values', connection, index=False)
tmp_params.to_sql('exp_parameters', connection, index=False)

CPU times: user 4.51 s, sys: 149 ms, total: 4.65 s
Wall time: 4.65 s


In [15]:
!du -sh /tmp/dgemm*

250M	/tmp/dgemm.csv
159M	/tmp/dgemm.db


We see that moving the parameters in their own table helped a bit, good.

In [16]:
%time tmp = pandas.read_sql('select * from exp_values inner join exp_parameters on exp_values.parameters_id = exp_parameters.parameters_id', connection)
print(len(tmp))
tmp.head()

CPU times: user 14.3 s, sys: 1.52 s, total: 15.8 s
Wall time: 15.8 s
5004288


,timestamp,duration,parameters_id,function,m,n,k,node,core,parameters_id
0,3723.065742,1.432000e-06,451899,dgemm,1,1,1,1,0,451899
1,3723.065746,2.530000e-07,451899,dgemm,1,1,1,1,0,451899
2,3723.065748,1.830000e-07,451899,dgemm,1,1,1,1,0,451899
3,3906.877902,1.520000e-06,451899,dgemm,1,1,1,1,0,451899
4,3906.877906,2.890000e-07,451899,dgemm,1,1,1,1,0,451899


Sadly, reading the table takes longer...

In [17]:
%time tmp = pandas.read_sql('select duration from exp_values inner join exp_parameters on exp_values.parameters_id = exp_parameters.parameters_id', connection)
print(len(tmp))
tmp.head()

CPU times: user 3.33 s, sys: 450 ms, total: 3.78 s
Wall time: 3.78 s
5004288


,duration
0,1.432000e-06
1,2.530000e-07
2,1.830000e-07
3,1.520000e-06
4,2.890000e-07


In [18]:
%time tmp = pandas.read_sql('select * from exp_values inner join exp_parameters on exp_values.parameters_id = exp_parameters.parameters_id where node == 20', connection)
print(len(tmp))
tmp.head()

CPU times: user 3.93 s, sys: 2.83 s, total: 6.76 s
Wall time: 6.78 s
156384


,timestamp,duration,parameters_id,function,m,n,k,node,core,parameters_id
0,3485.569752,0.486023,191136,dgemm,378,7640,2427,20,0,191136
1,3486.055798,0.486281,191136,dgemm,378,7640,2427,20,0,191136
2,3486.542086,0.485554,191136,dgemm,378,7640,2427,20,0,191136
3,4593.538610,0.486360,191136,dgemm,378,7640,2427,20,0,191136
4,4594.024977,0.486772,191136,dgemm,378,7640,2427,20,0,191136


Wow, these durations are crazy. I must be doing something wrong.

## A tentative to shrink the database: using keys and index

Note: we cannot do the table creation directly with Pandas, we have to run the SQL queries ourselves.

In [19]:
!rm /tmp/dgemm.db

In [20]:
connection = sqlite3.connect('/tmp/dgemm.db')
cursor = connection.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS "exp_parameters" (
"function" TEXT,
  m INTEGER,
  n INTEGER,
  k INTEGER,
  node INTEGER,
  core INTEGER,
  parameters_id INTEGER PRIMARY KEY
)''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS "exp_values" (
  timestamp REAL,
  duration REAL,
  parameters_id INTEGER,
  FOREIGN KEY(parameters_id) REFERENCES exp_parameters(parameters_id)
)''')
cursor.execute('''
CREATE INDEX parameters_index
    ON exp_values(parameters_id)
''')

In [21]:
%%time
connection = sqlite3.connect('/tmp/dgemm.db')
tmp_values.to_sql('exp_values', connection, index=False, if_exists='append')
tmp_params.to_sql('exp_parameters', connection, index=False, if_exists='append')

CPU times: user 7.6 s, sys: 2.15 s, total: 9.75 s
Wall time: 9.75 s


In [22]:
!du -sh /tmp/dgemm*

250M	/tmp/dgemm.csv
221M	/tmp/dgemm.db


In [23]:
%time tmp = pandas.read_sql('select * from exp_values inner join exp_parameters on exp_values.parameters_id = exp_parameters.parameters_id', connection)
print(len(tmp))
tmp.head()

CPU times: user 12.9 s, sys: 1.56 s, total: 14.5 s
Wall time: 14.5 s
5004288


,timestamp,duration,parameters_id,function,m,n,k,node,core,parameters_id
0,3723.065742,1.432000e-06,451899,dgemm,1,1,1,1,0,451899
1,3723.065746,2.530000e-07,451899,dgemm,1,1,1,1,0,451899
2,3723.065748,1.830000e-07,451899,dgemm,1,1,1,1,0,451899
3,3906.877902,1.520000e-06,451899,dgemm,1,1,1,1,0,451899
4,3906.877906,2.890000e-07,451899,dgemm,1,1,1,1,0,451899


In [24]:
%time tmp = pandas.read_sql('select * from exp_values inner join exp_parameters on exp_values.parameters_id = exp_parameters.parameters_id where node == 20', connection)
print(len(tmp))
tmp.head()

CPU times: user 451 ms, sys: 26.9 ms, total: 478 ms
Wall time: 477 ms
156384


,timestamp,duration,parameters_id,function,m,n,k,node,core,parameters_id
0,3485.569752,0.486023,191136,dgemm,378,7640,2427,20,0,191136
1,3486.055798,0.486281,191136,dgemm,378,7640,2427,20,0,191136
2,3486.542086,0.485554,191136,dgemm,378,7640,2427,20,0,191136
3,4593.538610,0.486360,191136,dgemm,378,7640,2427,20,0,191136
4,4594.024977,0.486772,191136,dgemm,378,7640,2427,20,0,191136


This is better, but we are still worse than the single-table case when we select everything. Also, note the important increase of storage space.